# Grfns in Julia

Here is an example of how to represent a grfn from https://github.com/ml4ai/delphi
specifically http://vision.cs.arizona.edu/adarsh/Delphi-Demo-Notebook.html.

We show how to build a Julia representation of the GrFN intermediate representation.

In [111]:
# you only need to install these dependencies once
using Pkg
try
    using JSON2
catch
    Pkg.add(["JSON2"])
end

The GrFN can be serialized to JSON. We will want to dispatch on the named tuple representation that is produced by JSON2 instead of the original JSON package which uses Dictionaries.

In [2]:
using JSON2

## Example Crop Model

Here is the JSON representation of a crop model produced by Delphi.

In [112]:
testgrfn = """{
"start": "CROP_YIELD",
"functions": [
  {
    "name": "UPDATE_EST__assign__TOTAL_RAIN_0",
    "type": "assign",
    "target": "TOTAL_RAIN",
    "sources": [
      "TOTAL_RAIN",
      "RAIN"
    ],
    "body": [
      {
        "type": "lambda",
        "name": "UPDATE_EST__lambda__TOTAL_RAIN_0",
        "reference": 4
      }
    ]
  },
  {
    "name": "UPDATE_EST__condition__IF_1_0",
    "type": "assign",
    "target": "IF_1",
    "sources": [
      "TOTAL_RAIN"
    ],
    "body": [
      {
        "type": "lambda",
        "name": "UPDATE_EST__lambda__IF_1_0",
        "reference": 5
      }
    ]
  },
  {
    "name": "UPDATE_EST__assign__YIELD_EST_0",
    "type": "assign",
    "target": "YIELD_EST",
    "sources": [
      "TOTAL_RAIN"
    ],
    "body": [
      {
        "type": "lambda",
        "name": "UPDATE_EST__lambda__YIELD_EST_0",
        "reference": 6
      }
    ]
  },
  {
    "name": "UPDATE_EST__assign__YIELD_EST_1",
    "type": "assign",
    "target": "YIELD_EST",
    "sources": [
      "TOTAL_RAIN"
    ],
    "body": [
      {
        "type": "lambda",
        "name": "UPDATE_EST__lambda__YIELD_EST_1",
        "reference": 8
      }
    ]
  },
  {
    "name": "UPDATE_EST__decision__YIELD_EST_0",
    "type": "assign",
    "target": "YIELD_EST",
    "sources": [
      "IF_1_0",
      "YIELD_EST_2",
      "YIELD_EST_1"
    ]
  },
  {
    "name": "UPDATE_EST",
    "type": "container",
    "input": [
      {
        "name": "RAIN",
        "domain": "real"
      },
      {
        "name": "TOTAL_RAIN",
        "domain": "real"
      },
      {
        "name": "YIELD_EST",
        "domain": "real"
      }
    ],
    "variables": [
      {
        "name": "TOTAL_RAIN",
        "domain": "real"
      },
      {
        "name": "RAIN",
        "domain": "real"
      },
      {
        "name": "IF_1",
        "domain": "boolean"
      },
      {
        "name": "YIELD_EST",
        "domain": "real"
      }
    ],
    "body": [
      {
        "name": "UPDATE_EST__assign__TOTAL_RAIN_0",
        "output": {
          "variable": "TOTAL_RAIN",
          "index": 1
        },
        "input": [
          {
            "variable": "TOTAL_RAIN",
            "index": 0
          },
          {
            "variable": "RAIN",
            "index": 0
          }
        ]
      },
      {
        "name": "UPDATE_EST__condition__IF_1_0",
        "output": {
          "variable": "IF_1",
          "index": 0
        },
        "input": [
          {
            "variable": "TOTAL_RAIN",
            "index": 1
          }
        ]
      },
      {
        "name": "UPDATE_EST__assign__YIELD_EST_0",
        "output": {
          "variable": "YIELD_EST",
          "index": 1
        },
        "input": [
          {
            "variable": "TOTAL_RAIN",
            "index": 1
          }
        ]
      },
      {
        "name": "UPDATE_EST__assign__YIELD_EST_1",
        "output": {
          "variable": "YIELD_EST",
          "index": 2
        },
        "input": [
          {
            "variable": "TOTAL_RAIN",
            "index": 1
          }
        ]
      },
      {
        "name": "UPDATE_EST__decision__YIELD_EST_0",
        "output": {
          "variable": "YIELD_EST",
          "index": 3
        },
        "input": [
          {
            "variable": "IF_1",
            "index": 0
          },
          {
            "variable": "YIELD_EST",
            "index": 2
          },
          {
            "variable": "YIELD_EST",
            "index": 1
          }
        ]
      }
    ]
  },
  {
    "name": "CROP_YIELD__assign__MAX_RAIN_0",
    "type": "assign",
    "target": "MAX_RAIN",
    "sources": [],
    "body": {
      "type": "literal",
      "dtype": "real",
      "value": "4.0"
    }
  },
  {
    "name": "CROP_YIELD__assign__CONSISTENCY_0",
    "type": "assign",
    "target": "CONSISTENCY",
    "sources": [],
    "body": {
      "type": "literal",
      "dtype": "real",
      "value": "64.0"
    }
  },
  {
    "name": "CROP_YIELD__assign__ABSORPTION_0",
    "type": "assign",
    "target": "ABSORPTION",
    "sources": [],
    "body": {
      "type": "literal",
      "dtype": "real",
      "value": "0.6"
    }
  },
  {
    "name": "CROP_YIELD__assign__YIELD_EST_0",
    "type": "assign",
    "target": "YIELD_EST",
    "sources": [],
    "body": {
      "type": "literal",
      "dtype": "integer",
      "value": "0"
    }
  },
  {
    "name": "CROP_YIELD__assign__TOTAL_RAIN_0",
    "type": "assign",
    "target": "TOTAL_RAIN",
    "sources": [],
    "body": {
      "type": "literal",
      "dtype": "integer",
      "value": "0"
    }
  },
  {
    "name": "CROP_YIELD__assign__RAIN_0",
    "type": "assign",
    "target": "RAIN",
    "sources": [
      "DAY",
      "CONSISTENCY",
      "MAX_RAIN",
      "ABSORPTION"
    ],
    "body": [
      {
        "type": "lambda",
        "name": "CROP_YIELD__lambda__RAIN_0",
        "reference": 24
      }
    ]
  },
  {
    "name": "CROP_YIELD__loop_plate__DAY_0",
    "type": "loop_plate",
    "input": [
      "CONSISTENCY",
      "MAX_RAIN",
      "ABSORPTION",
      "RAIN",
      "TOTAL_RAIN",
      "YIELD_EST"
    ],
    "index_variable": "DAY",
    "index_iteration_range": {
      "start": {
        "type": "literal",
        "dtype": "integer",
        "value": 1
      },
      "end": {
        "value": 32,
        "dtype": "integer",
        "type": "literal"
      }
    },
    "body": [
      {
        "name": "CROP_YIELD__assign__RAIN_0",
        "output": {
          "variable": "RAIN",
          "index": 0
        },
        "input": [
          {
            "variable": "DAY",
            "index": -1
          },
          {
            "variable": "CONSISTENCY",
            "index": -1
          },
          {
            "variable": "MAX_RAIN",
            "index": -1
          },
          {
            "variable": "ABSORPTION",
            "index": -1
          }
        ]
      },
      {
        "function": "UPDATE_EST",
        "output": {},
        "input": [
          {
            "variable": "RAIN",
            "index": 0
          },
          {
            "variable": "TOTAL_RAIN",
            "index": -1
          },
          {
            "variable": "YIELD_EST",
            "index": -1
          }
        ]
      },
      {
        "function": "print",
        "output": {},
        "input": [
          {
            "variable": "DAY",
            "index": -1
          },
          {
            "variable": "YIELD_EST",
            "index": -1
          }
        ]
      }
    ]
  },
  {
    "name": "CROP_YIELD",
    "type": "container",
    "input": [],
    "variables": [
      {
        "name": "DAY",
        "domain": "integer"
      },
      {
        "name": "RAIN",
        "domain": "real"
      },
      {
        "name": "YIELD_EST",
        "domain": "real"
      },
      {
        "name": "TOTAL_RAIN",
        "domain": "real"
      },
      {
        "name": "MAX_RAIN",
        "domain": "real"
      },
      {
        "name": "CONSISTENCY",
        "domain": "real"
      },
      {
        "name": "ABSORPTION",
        "domain": "real"
      }
    ],
    "body": [
      {
        "name": "CROP_YIELD__assign__MAX_RAIN_0",
        "output": {
          "variable": "MAX_RAIN",
          "index": 2
        },
        "input": []
      },
      {
        "name": "CROP_YIELD__assign__CONSISTENCY_0",
        "output": {
          "variable": "CONSISTENCY",
          "index": 2
        },
        "input": []
      },
      {
        "name": "CROP_YIELD__assign__ABSORPTION_0",
        "output": {
          "variable": "ABSORPTION",
          "index": 2
        },
        "input": []
      },
      {
        "name": "CROP_YIELD__assign__YIELD_EST_0",
        "output": {
          "variable": "YIELD_EST",
          "index": 2
        },
        "input": []
      },
      {
        "name": "CROP_YIELD__assign__TOTAL_RAIN_0",
        "output": {
          "variable": "TOTAL_RAIN",
          "index": 2
        },
        "input": []
      },
      {
        "name": "CROP_YIELD__loop_plate__DAY_0",
        "inputs": [
          "CONSISTENCY",
          "MAX_RAIN",
          "ABSORPTION",
          "RAIN",
          "TOTAL_RAIN",
          "YIELD_EST"
        ],
        "output": {}
      },
      {
        "function": "print",
        "output": {},
        "input": [
          {
            "variable": "YIELD_EST",
            "index": 2
          }
        ]
      }
    ]
  }
],
"name": "pgm.json",
"dateCreated": "2018-10-04"
}
""";


## GrFN datastructures

Here we define the data structures that will represent the nodes of the GrFN. In particular notice the `Grfn`, `GrfnNode`, `Assignment`, `LoopPlate`, and `Container` types. These are used in the example.

In [113]:
module GrfnMod

mutable struct Grfn{D,T}
    name::String
    start::String
    created::D
    functions::T
end
struct Input
    name::String
    domain::String
end

struct Container{I,V,B}
    name::String
    type::String
    input::I
    variables::V
    body::B
end
Container(tuple) = Container(tuple...)

mutable struct GrfnBody{S,T,U}
    name::S
    type::T
    reference::U
end

mutable struct GrfnSource{S}
    name::S
end
    
mutable struct GrfnNode{T,U,V}
    name::String
    type::T
    target::String
    sources::V
    body::U
end
#GrfnSource(x::NamedTuple) = GrfnSource(x...)
GrfnBody(x::NamedTuple) = GrfnBody(x...)
# GrfnNode(n,t,r,s::Vector{Any},b::Vector{Any}) = begin
#     GrfnNode(n,t,r,GrfnSource.(s), GrfnBody.(b))
# end
struct Assignment{T,U}
    name::String
    typ::String
    target::String
    sources::T
    body::U
end
function Assignment(n,t,r,s)
    return Assignment(n,t,r,s,Missing)
end

function Assignment(tuple::NamedTuple)
    print(tuple)
    return Assignment(tuple...)
end

struct LoopPlate{I,T,R,B}
    name::String
    type::String
    input::I
    index_variable::T
    index_iteration_range::R
    body::B                
end
LoopPlate(t::NamedTuple) = LoopPlate(t...)
end

Main.GrfnMod

We can read in the raw JSON into a hierarchical representation. This is very similar to the pythonic way where you have a fully dynamic representation. We have a little more typing information because we have `NamedTuples{K,V}` where `K` is the names of the keys and `V` is the types of the values in the tuple. This representation lets us use multiple dispatch when implementing our GrFN to Julia expression functions.

In [118]:
sobj = JSON2.read(testgrfn);

We can interrogate the parsed data structure to make sure it matches what we expect.

In [119]:
sobj.start

"CROP_YIELD"

In [128]:
sobj.functions[3:6]

4-element Array{Any,1}:
 (name = "UPDATE_EST__assign__YIELD_EST_0", type = "assign", target = "YIELD_EST", sources = Any["TOTAL_RAIN"], body = Any[(type = "lambda", name = "UPDATE_EST__lambda__YIELD_EST_0", reference = 6)])                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

### Constructing the Grfn object
Once we have it parsed, we can convert it into the julia types we defined above. `NamedTuples` are autogenerated types which allow us to have performant IO but are harder to work with than manually created structs.

In [129]:
@show sobj.name
@show sobj.dateCreated
@show sobj.start

nodes = []
for func in sobj.functions
    println("========================")
    if func.type == "assign"
      node = GrfnMod.Assignment(func)
    elseif func.type == "container"
      println("Found a Container Node")
      node = GrfnMod.Container(func)
    elseif func.type == "loop_plate"
      println("Found a LoopPlate Node")
      node = GrfnMod.LoopPlate(func)
    else
      @warn "Found an unknown node type"
      dump(func)
    end
    push!(nodes, node)
    println("------------------------")
end

sobj.name = "pgm.json"
sobj.dateCreated = "2018-10-04"
sobj.start = "CROP_YIELD"
(name = "UPDATE_EST__assign__TOTAL_RAIN_0", type = "assign", target = "TOTAL_RAIN", sources = Any["TOTAL_RAIN", "RAIN"], body = Any[(type = "lambda", name = "UPDATE_EST__lambda__TOTAL_RAIN_0", reference = 4)])------------------------
(name = "UPDATE_EST__condition__IF_1_0", type = "assign", target = "IF_1", sources = Any["TOTAL_RAIN"], body = Any[(type = "lambda", name = "UPDATE_EST__lambda__IF_1_0", reference = 5)])------------------------
(name = "UPDATE_EST__assign__YIELD_EST_0", type = "assign", target = "YIELD_EST", sources = Any["TOTAL_RAIN"], body = Any[(type = "lambda", name = "UPDATE_EST__lambda__YIELD_EST_0", reference = 6)])------------------------
(name = "UPDATE_EST__assign__YIELD_EST_1", type = "assign", target = "YIELD_EST", sources = Any["TOTAL_RAIN"], body = Any[(type = "lambda", name = "UPDATE_EST__lambda__YIELD_EST_1", reference = 8)])------------------------
(name = "UPDATE_EST__decisio

In [130]:
g = GrfnMod.Grfn(sobj.name, sobj.start, sobj.dateCreated, nodes)

Main.GrfnMod.Grfn{String,Array{Any,1}}("pgm.json", "CROP_YIELD", "2018-10-04", Any[Assignment{Array{Any,1},Array{Any,1}}("UPDATE_EST__assign__TOTAL_RAIN_0", "assign", "TOTAL_RAIN", Any["TOTAL_RAIN", "RAIN"], Any[(type = "lambda", name = "UPDATE_EST__lambda__TOTAL_RAIN_0", reference = 4)]), Assignment{Array{Any,1},Array{Any,1}}("UPDATE_EST__condition__IF_1_0", "assign", "IF_1", Any["TOTAL_RAIN"], Any[(type = "lambda", name = "UPDATE_EST__lambda__IF_1_0", reference = 5)]), Assignment{Array{Any,1},Array{Any,1}}("UPDATE_EST__assign__YIELD_EST_0", "assign", "YIELD_EST", Any["TOTAL_RAIN"], Any[(type = "lambda", name = "UPDATE_EST__lambda__YIELD_EST_0", reference = 6)]), Assignment{Array{Any,1},Array{Any,1}}("UPDATE_EST__assign__YIELD_EST_1", "assign", "YIELD_EST", Any["TOTAL_RAIN"], Any[(type = "lambda", name = "UPDATE_EST__lambda__YIELD_EST_1", reference = 8)]), Assignment{Array{Any,1},DataType}("UPDATE_EST__decision__YIELD_EST_0", "assign", "YIELD_EST", Any["IF_1_0", "YIELD_EST_2", "YIELD_

In [131]:
for n in g.functions[3:6]
    println(n)
    println()
end


Main.GrfnMod.Assignment{Array{Any,1},Array{Any,1}}("UPDATE_EST__assign__YIELD_EST_0", "assign", "YIELD_EST", Any["TOTAL_RAIN"], Any[(type = "lambda", name = "UPDATE_EST__lambda__YIELD_EST_0", reference = 6)])

Main.GrfnMod.Assignment{Array{Any,1},Array{Any,1}}("UPDATE_EST__assign__YIELD_EST_1", "assign", "YIELD_EST", Any["TOTAL_RAIN"], Any[(type = "lambda", name = "UPDATE_EST__lambda__YIELD_EST_1", reference = 8)])

Main.GrfnMod.Assignment{Array{Any,1},DataType}("UPDATE_EST__decision__YIELD_EST_0", "assign", "YIELD_EST", Any["IF_1_0", "YIELD_EST_2", "YIELD_EST_1"], Missing)

Main.GrfnMod.Container{Array{Any,1},Array{Any,1},Array{Any,1}}("UPDATE_EST", "container", Any[(name = "RAIN", domain = "real"), (name = "TOTAL_RAIN", domain = "real"), (name = "YIELD_EST", domain = "real")], Any[(name = "TOTAL_RAIN", domain = "real"), (name = "RAIN", domain = "real"), (name = "IF_1", domain = "boolean"), (name = "YIELD_EST", domain = "real")], Any[(name = "UPDATE_EST__assign__TOTAL_RAIN_0", output 

## Converting to Julia Expressions

There are two ways to execute a model described at the semantic level. We could develop an interpreter to traverse the model and execute it directly, or we can compiler the model into a general purpose programming language and then run it. 

We want to convert the representation into Julia Expressions so that the we can use the Julia Machinery to execute it just like hand written code.

In [186]:
"""    typename(s)

map a fortran type to a Julia type (ie. real -> Float64, integer -> Int)
"""
function typename(s::Symbol)
    if s == :integer
        return :Int
    elseif s == :real
        return :Float64
    else
        return s
    end
end
typename(s::String) = typename(Symbol(s))
literal(t::NamedTuple) = :($(Symbol(t.value))::$(typename(t.dtype)))
range(t::NamedTuple) = :($(t.start.value):$(t.end.value)::$(typename(t.end.dtype)))
function lower(n::GrfnMod.Assignment)
    #show(n)
    if n.body==Missing
        #@warn "missing body"
        return :($(Symbol(n.target)) = :Missing)
    end
    lhs = Symbol(n.target)
    if isa(n.body, NamedTuple)
        body = n.body
        rhs = literal(body) #:($(body.value)::$(getfield(body,:dtype)))
        return :($lhs = $rhs)
    else
        body = n.body[1]
    end
    if body == "literal"
        rhs = body.value.value
    else
        rhs = Symbol(body.name)
    end
    return :($lhs = $rhs)
end

"""    lower(b::NamedTuple{(:variable,:index),T})

extracts a variable reference into an Expr
"""
function lower(b::NamedTuple{(:variable,:index),T}) where T
    ex = :( Var($(Symbol(b.variable)),$(b.index)))
    return ex
end

# TODO why does this node type exist?
function lower(b::NamedTuple{(:name, :output, :input),T}) where T
    ex = :($(lower(b.output)) = $(Symbol(b.name))($(lower.(b.input)...)))
    return ex
end

"""    lower(b::NamedTuple{(:function, :output, :input),T})

extracts a function call into an Expr
"""
function lower(b::NamedTuple{(:function, :output, :input),T}) where T
    args = lower(b.input)
    ex = :($(Symbol(b.function))($(args...)))
    if length(b.output) >= 1
        ex = :($(b.output) = $ex)
    end
    return ex
end
                
"""    lower(b::Vector{T})

extracts a loop body into an Expr
"""       
function lower(b::Vector{T}) where T
    items = Expr[]                 
    for x in b
        ex = lower(x)
        push!(items, ex)
    end
    return items            
end
function lower(b::NamedTuple{(:name, :inputs, :output),
                        Tuple{String,
                            Array{Any,1},
                            NamedTuple{(),Tuple{}}}})  
    ex = :(Plate($(Symbol(b.name)), $(Symbol.(b.inputs)...)))
    return ex                
end
                
function lower(n::GrfnMod.LoopPlate)
    name, input = Symbol(n.name), Symbol.(n.input)
    indexvar = Symbol(n.index_variable)
    indexrange = range(n.index_iteration_range)
    ex = :(Loop(name=$name,
                input=$(input),
                indexvar=$indexvar,
                range=$indexrange,
                            body=begin$(lower(n.body)...) end
        )
    )
    return ex
end

lower (generic function with 18 methods)

In [187]:
           
function lower(t::NamedTuple{(:name, :domain),T}) where T
    ex = :($(Symbol(t.name)) => $(Symbol(t.domain)))
    return ex
end

function lower(n::GrfnMod.Container)
    q = quote
    end
    for b in n.body
        try
            ex = lower(b)
            push!(q.args, ex)
        catch exception
            @warn exception
            dump(b)
        end
    end
    ex = :(Container($(Symbol(n.name)),
            domains=Dict($(lower.(n.input)...)),
            variables=Dict($(lower.(n.variables)...)),
            body=$q
        )
    )
    return ex
end  
# body = Expr[]
# for n in g.functions
# #     try
#         l = lower(n)
#         push!(body, l)
#         #println("---------------------------")
# #     catch ex
# #         println("======================")
# #         dump(n)
# #         println()
# #         println(ex)
# #         dump(ex)
# #         println()
# #         println("======================")
# #     end
    
# end

lower (generic function with 18 methods)

In [188]:
function lower(n::GrfnMod.Grfn)
    ex = :(Grfn($(n.name), $(n.created), $(n.start), begin $(lower(n.functions)...)end))
    return ex
end

lower (generic function with 18 methods)

### Lowering our example graph

We can get a julia expression from the graph. Which we could execute with `eval`.

In [189]:
gl = lower(g)

:(Grfn("pgm.json", "2018-10-04", "CROP_YIELD", begin
          #= In[188]:2 =#
          TOTAL_RAIN = UPDATE_EST__lambda__TOTAL_RAIN_0
          IF_1 = UPDATE_EST__lambda__IF_1_0
          YIELD_EST = UPDATE_EST__lambda__YIELD_EST_0
          YIELD_EST = UPDATE_EST__lambda__YIELD_EST_1
          YIELD_EST = :Missing
          Container(UPDATE_EST, domains=Dict(RAIN => real, TOTAL_RAIN => real, YIELD_EST => real), variables=Dict(TOTAL_RAIN => real, RAIN => real, IF_1 => boolean, YIELD_EST => real), body=begin
                      #= In[187]:9 =#
                      Var(TOTAL_RAIN, 1) = UPDATE_EST__assign__TOTAL_RAIN_0(Var(TOTAL_RAIN, 0), Var(RAIN, 0))
                      Var(IF_1, 0) = UPDATE_EST__condition__IF_1_0(Var(TOTAL_RAIN, 1))
                      Var(YIELD_EST, 1) = UPDATE_EST__assign__YIELD_EST_0(Var(TOTAL_RAIN, 1))
                      Var(YIELD_EST, 2) = UPDATE_EST__assign__YIELD_EST_1(Var(TOTAL_RAIN, 1))
                      Var(YIELD_EST, 3) = UPDATE_EST__decision_

In [190]:
dump(gl.args)

Array{Any}((5,))
  1: Symbol Grfn
  2: String "pgm.json"
  3: String "2018-10-04"
  4: String "CROP_YIELD"
  5: Expr
    head: Symbol block
    args: Array{Any}((15,))
      1: LineNumberNode
        line: Int64 2
        file: Symbol In[188]
      2: Expr
        head: Symbol =
        args: Array{Any}((2,))
          1: Symbol TOTAL_RAIN
          2: Symbol UPDATE_EST__lambda__TOTAL_RAIN_0
      3: Expr
        head: Symbol =
        args: Array{Any}((2,))
          1: Symbol IF_1
          2: Symbol UPDATE_EST__lambda__IF_1_0
      4: Expr
        head: Symbol =
        args: Array{Any}((2,))
          1: Symbol YIELD_EST
          2: Symbol UPDATE_EST__lambda__YIELD_EST_0
      5: Expr
        head: Symbol =
        args: Array{Any}((2,))
          1: Symbol YIELD_EST
          2: Symbol UPDATE_EST__lambda__YIELD_EST_1
      ...
      11: Expr
        head: Symbol =
        args: Array{Any}((2,))
          1: Symbol YIELD_EST
          2: Expr
            head: Symbol ::
          

Ideally, we would eval this expression which would build a Julia object representing the model at the semantic level and then have a function called `run` or `solve` which would execute the model and produce a Julia object representing the result. By representing the model in regular Julia code, we have the benefit of a general purpose programming language. By building a hierarchical representation of the model, we get the benefits of being able to perform high level transformations on the models before we execute them.

TODOs:

1. Figure out how to deal with the references and indexes
2. Attach to the lower level functions to actually execute the model.
3. Write model transformations on the Expr or DAG level.